# Probability Search

In [1]:
from __future__ import division
from __future__ import print_function

# Allowed libraries 
import numpy as np
import pandas as pd
import scipy as sp
import scipy.special
import heapq as pq
import matplotlib as mp
import matplotlib.pyplot as plt
import math
from itertools import product, combinations
from collections import OrderedDict as odict
import collections
from graphviz import Digraph, Graph
from tabulate import tabulate
import copy
import sys
import os
import datetime
import sklearn
import ast
import re

In [2]:
data = pd.read_csv("data.csv")

In [3]:
data.head(5)

,Unnamed: 0,reliable_sensor1,reliable_sensor2,reliable_sensor3,reliable_sensor4,unreliable_sensor1,unreliable_sensor2,unreliable_sensor3,unreliable_sensor4,robot1,robot2,door_sensor1,door_sensor2,door_sensor3,door_sensor4,time,electricity_price,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13,r14,r15,r16,r17,r18,r19,r20,r21,r22,r23,r24,r25,r26,r27,r28,r29,r30,r31,r32,r33,r34,r35,c1,c2,c3,c4,o1,outside
0,0,no motion,no motion,no motion,no motion,no motion,motion,no motion,no motion,"('r1', 0)","('r19', 0)",0,0,0,0,08:00:15,0.9800,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17
1,1,no motion,no motion,motion,no motion,no motion,no motion,no motion,no motion,"('r1', 0)","('r19', 0)",0,0,0,0,08:00:30,0.9604,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,15
2,2,no motion,no motion,motion,no motion,no motion,no motion,no motion,no motion,"('r1', 0)","('r19', 0)",0,0,0,0,08:00:45,0.9800,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,8,0,0,4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,3,no motion,no motion,motion,no motion,no motion,motion,no motion,no motion,"('r1', 0)","('c3', 0)",0,0,0,0,08:01:00,0.9604,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,8,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0
4,4,no motion,no motion,motion,no motion,no motion,no motion,no motion,no motion,"('r1', 0)","('r20', 0)",0,2,0,0,08:01:15,0.9604,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,8,0,0,0,0,0,0,0,0,0,0,11,2,0,0,0,0


In [7]:
r12_not_empty_idx = np.where(data['r12']>0)
data.iloc[r12_not_empty_idx]['time']

0       08:00:15
1       08:00:30
2       08:00:45
909     11:47:30
910     11:47:45
          ...   
2318    17:39:45
2319    17:40:00
2320    17:40:15
2321    17:40:30
2322    17:40:45
Name: time, Length: 179, dtype: object

In [1]:
from graphviz import Graph


## The correctness of Reliable Sensors

Assumption: <br>
1. In 15s, a person will not come in and leave that room from the same entrance.<br>
2. One person can at most pass 2 areas (including rooms, outside etc.) in 15s. <br>
3. Assume people never detour to their destinations. <br>

In [10]:
data.shape

(2401, 58)

### Reliable Sensor 1

In [17]:
wrong_count = 0
for i in range(data.shape[0]):
    if (i==0):
        if (data.iloc[0]['r16']>0):
            wrong_count +=1
    if (i>0):
        curr_row = data.iloc[i] 
        prev_row = data.iloc[i-1]
        if(curr_row['r16']== 0 and prev_row['r16']== 0):
            if (curr_row['reliable_sensor1']=='motion'):
                wrong_count +=1 
        elif(curr_row['r16']>0 and curr_row['reliable_sensor1']=='no motion'):
            wrong_count +=1
        elif(curr_row['r16']==0 and prev_row['r16']>0):
            if (curr_row['reliable_sensor1']=='no motion'):
                wrong_count +=1
        
print(wrong_count)
print("The correction probability is ",round(1-wrong_count/data.shape[0],4))

66
The correction probability is  0.9725


### Reliable Sensor 2

Situations that pass room 5 without staying within 15s: <br>
1. 9-5-6 <br>
2. 6-5-9 <br>
3. c3-5-9 <br>
4. 9-5-c3 <br>


In [19]:
wrong_count = 0
for i in range(data.shape[0]):
    if (i==0):
        if (data.iloc[0]['r5']>0):
            wrong_count +=1
    if (i>0):
        curr_row = data.iloc[i] 
        prev_row = data.iloc[i-1]
        # if a person stays in r5, the motion should be true
        if(curr_row['r5']>0 and curr_row['reliable_sensor2']=='no motion'):
            wrong_count +=1
        # if sensor detects motion, but r5 is empty, then there must be some one passes by
        if (curr_row['reliable_sensor2']=='motion' and curr_row['r5']==0):
            flag = True # flag indicating whether reliable sensor goes wrong
            if (prev_row['r9']>0 and curr_row['r6']>0): # someone can go from r9 to r6
                flag = False 
            elif (prev_row['r6']>0 and curr_row['r9']>0):
                flag = False 
            elif (prev_row['c3']>0 and curr_row['r9']>0):
                flag = False 
            elif (prev_row['r9']>0 and curr_row['c3']>0):
                flag = False 
            
            if (flag == True):
                wrong_count += 1 
        
print(wrong_count)
print("The correction probability is ",round(1-wrong_count/data.shape[0],4))

68
The correction probability is  0.9717


### Reliable Sensor 3

Situations that pass room 25 without staying within 15s: <br>
1. 22-25-26 <br>
2. 26-25-22 <br>
3. c1-25-22 <br>
4. 22-25-c1 <br>
5. 26-25-c1 <br>
6. c1-25-26 <br>

In [20]:
wrong_count = 0
for i in range(data.shape[0]):
    if (i==0):
        if (data.iloc[0]['r25']>0):
            wrong_count +=1
    if (i>0):
        curr_row = data.iloc[i] 
        prev_row = data.iloc[i-1]
        # if a person stays in r25, the motion should be true
        if(curr_row['r25']>0 and curr_row['reliable_sensor3']=='no motion'):
            wrong_count +=1
        # if sensor detects motion, but r25 is empty, then there must be some one passes by
        if (curr_row['reliable_sensor3']=='motion' and curr_row['r25']==0):
            flag = True # flag indicating whether reliable sensor goes wrong
            if (prev_row['r22']>0 and curr_row['r26']>0): 
                flag = False 
            elif (prev_row['r26']>0 and curr_row['r22']>0):
                flag = False 
            elif (prev_row['c1']>0 and curr_row['r22']>0):
                flag = False 
            elif (prev_row['r22']>0 and curr_row['c1']>0):
                flag = False 
            elif (prev_row['r26']>0 and curr_row['c1']>0):
                flag = False 
            elif (prev_row['c1']>0 and curr_row['r26']>0):
                flag = False 
            
            if (flag == True):
                wrong_count += 1 
        
print(wrong_count)
print("The correction probability is ",round(1-wrong_count/data.shape[0],4))

59
The correction probability is  0.9754


### Reliable Sensor 4

In [4]:
wrong_count = 0
for i in range(data.shape[0]):
    if (i==0):
        if (data.iloc[0]['r31']>0):
            wrong_count +=1
    if (i>0):
        curr_row = data.iloc[i] 
        prev_row = data.iloc[i-1]
        # if a person stays in r25, the motion should be true
        if(curr_row['r31']>0 and curr_row['reliable_sensor4']=='no motion'):
            wrong_count +=1
        # if sensor detects motion, but r25 is empty, then there must be some one passes by
        if (curr_row['reliable_sensor4']=='motion' and curr_row['r31']==0):
            wrong_count += 1 
        
print(wrong_count)
print("The correction probability is ",round(1-wrong_count/data.shape[0],4))

68
The correction probability is  0.9717


### Unreliable Sensor 1

In [5]:
wrong_count = 0
for i in range(data.shape[0]):
    if (i==0):
        if (data.iloc[0]['o1']>0):
            wrong_count +=1
    if (i>0):
        curr_row = data.iloc[i] 
        prev_row = data.iloc[i-1]
        # if a person stays in o1, the motion should be true
        if(curr_row['o1']>0 and curr_row['unreliable_sensor1']=='no motion'):
            wrong_count +=1
        # if sensor detects motion, but o1 is empty, then there must be some one passes by
        if (curr_row['unreliable_sensor1']=='motion' and curr_row['o1']==0):
            flag = True # flag indicating whether reliable sensor goes wrong
            if (prev_row['c3']>0 and curr_row['c4']>0): 
                flag = False 
            elif (prev_row['c4']>0 and curr_row['c3']>0):
                flag = False 
            
            if (flag == True):
                wrong_count += 1 
        
print(wrong_count)
print("The correction probability is ",round(1-wrong_count/data.shape[0],4))

164
The correction probability is  0.9317


In [5]:
cand = ['r5','r6','r10','r11','r15','r16','r17','r18','r19','r20','r21','o1']

### Unreliable Sensor 2

In [8]:
wrong_count = 0
for i in range(data.shape[0]):
    if (i==0):
        if (data.iloc[0]['c3']>0):
            wrong_count +=1
    if (i>0):
        curr_row = data.iloc[i] 
        prev_row = data.iloc[i-1]
        # if a person stays in o1, the motion should be true
        if(curr_row['c3']>0 and curr_row['unreliable_sensor2']=='no motion'):
            wrong_count +=1
        # if sensor detects motion, but o1 is empty, then there must be some one passes by
        if (curr_row['unreliable_sensor2']=='motion' and curr_row['c3']==0):
            candidates = ['r5','r6','r10','r11','r15','r16','r17','r18','r19','r20','r21','o1']
            flag = True # flag indicating whether reliable sensor goes wrong
            for cand1 in candidates:
                for cand2 in candidates:
                    if (cand1==cand2):
                        continue 
                    if (prev_row[cand1]>0 and curr_row[cand2]>0):
                        flag = False
                        break
            
            if (flag == True):
                wrong_count += 1 
        
print(wrong_count)
print("The correction probability is ",round(1-wrong_count/data.shape[0],4))

101
The correction probability is  0.9579


### Unreliable Sensor 3

In [9]:
wrong_count = 0
for i in range(data.shape[0]):
    if (i==0):
        if (data.iloc[0]['r1']>0):
            wrong_count +=1
    if (i>0):
        curr_row = data.iloc[i] 
        prev_row = data.iloc[i-1]
        # if a person stays in o1, the motion should be true
        if(curr_row['r1']>0 and curr_row['unreliable_sensor3']=='no motion'):
            wrong_count +=1
        # if sensor detects motion, but o1 is empty, then there must be some one passes by
        if (curr_row['unreliable_sensor3']=='motion' and curr_row['r1']==0):
            candidates = ['r2','r3']
            flag = True # flag indicating whether reliable sensor goes wrong
            for cand1 in candidates:
                for cand2 in candidates:
                    if (cand1==cand2):
                        continue 
                    if (prev_row[cand1]>0 and curr_row[cand2]>0):
                        flag = False
                        break
            
            if (flag == True):
                wrong_count += 1 
        
print(wrong_count)
print("The correction probability is ",round(1-wrong_count/data.shape[0],4))

201
The correction probability is  0.9163


### Unreliable Sensor 4

In [10]:
wrong_count = 0
for i in range(data.shape[0]):
    if (i==0):
        if (data.iloc[0]['r24']>0):
            wrong_count +=1
    if (i>0):
        curr_row = data.iloc[i] 
        prev_row = data.iloc[i-1]
        # if a person stays in o1, the motion should be true
        if(curr_row['r24']>0 and curr_row['unreliable_sensor4']=='no motion'):
            wrong_count +=1
        # if sensor detects motion, but o1 is empty, then there must be some one passes by
        if (curr_row['unreliable_sensor4']=='motion' and curr_row['r24']==0):
            candidates = ['r23','r13','r14']
            flag = True # flag indicating whether reliable sensor goes wrong
            for cand1 in candidates:
                for cand2 in candidates:
                    if (cand1==cand2):
                        continue 
                    if (prev_row[cand1]>0 and curr_row[cand2]>0):
                        flag = False
                        break
            
            if (flag == True):
                wrong_count += 1 
        
print(wrong_count)
print("The correction probability is ",round(1-wrong_count/data.shape[0],4))

289
The correction probability is  0.8796
